## K-Core Sampling

- process files line by line and not try to read the whole thing into pandas. 
- Write four scripts, two each for users and items. 

1. Script A read the file line by line and output a table of users and their frequencies.
- Using the output of Script A, Script B reads the file again line by line and filters out the users with less than k items. 
2. Scripts C and D are similar to A and B, but for items. 
- Then you can run them A, B, C, D until the size of the file doesn't change. This should be relatively efficient.

In [92]:
# quickly make a copy of original data so I don't mess it up on accident!
# don't delete this cell
with open('final_track2_train.txt','r') as firstfile, open('k_core_ratings_0.txt','w') as secondfile:
    secondfile.truncate(0)
    # read content from first file
    for line in firstfile:
        # append content to second file
        secondfile.write(line)

In [102]:
#SCRIPT FOR CREATING K-CORE!!!
k = 80
i = 0

while i < 30:
    if i > 1:
        print("i is: ", i)
        previous_file_size = os.path.getsize("k_core_ratings_{0}.txt".format(i-2))
        print("Previous file size was :", previous_file_size, "bytes")
        current_file_size = os.path.getsize("k_core_ratings_{0}.txt".format(i))
        print("Current file size is :", current_file_size, "bytes")
        if previous_file_size - current_file_size < 100:
            print("files are close enough in size for convergence!")
            print("final file name: k_core_ratings_{0}.txt".format(i))
            break
    #Script A read the file line by line and output a table of users and their frequencies.
    user_freq = {}
    with open("k_core_ratings_{0}.txt".format(i)) as fp:
        for line in fp:
            features = line.split()
            user_id = features[0]
            if user_id in user_freq:
                user_freq[user_id] += 1
            else:
                user_freq[user_id] = 1

    # Using the output of Script A, Script B reads the file again line by line 
    # and filters out the users with less than k items.

    file1 = open("k_core_ratings_{0}.txt".format(i+1), 'w')
    filteredOut = 0
    with open("k_core_ratings_{0}.txt".format(i)) as fp:
        for line in fp:
            # Writing to file
            features = line.split()
            user_id = features[0]
            if user_freq[user_id] < k:
                filteredOut+=1
            else:
                file1.writelines(line)

    file1.close()

    #Scripts C and D are similar to A and B, but for items.
    item_freq = {}
    with open("k_core_ratings_{0}.txt".format(i+1)) as fp:
        for line in fp:
            features = line.split()
            item_id = features[2]
            if item_id in item_freq:
                item_freq[item_id] += 1
            else:
                item_freq[item_id] = 1

    file1 = open("k_core_ratings_{0}.txt".format(i+2), 'w')
    filteredOut = 0
    with open("k_core_ratings_{0}.txt".format(i+1)) as fp:
        for line in fp:
            # Writing to file
            features = line.split()
            item_id = features[2]
            if item_freq[item_id] < k:
                filteredOut+=1
            else:
                file1.writelines(line)

    file1.close()
    i+=2

i is:  2
Previous file size was : 1069889124 bytes
Current file size is : 400557525 bytes
i is:  4
Previous file size was : 400557525 bytes
Current file size is : 356604139 bytes
i is:  6
Previous file size was : 356604139 bytes
Current file size is : 352149539 bytes
i is:  8
Previous file size was : 352149539 bytes
Current file size is : 351579163 bytes
i is:  10
Previous file size was : 351579163 bytes
Current file size is : 351469487 bytes
i is:  12
Previous file size was : 351469487 bytes
Current file size is : 351448439 bytes
i is:  14
Previous file size was : 351448439 bytes
Current file size is : 351435587 bytes
i is:  16
Previous file size was : 351435587 bytes
Current file size is : 351435587 bytes
files are close enough in size for convergence!
final file name: k_core_ratings_16.txt


In [108]:
k_core_file = 16

In [109]:
#checking the length of the final dataset, and setting it for the creation of the
#different signal features
import pandas as pd
train_df = pd.read_csv("k_core_ratings_{0}.txt".format(k_core_file), sep='\t', names=['uid', 'user_city', 'item_id', 'author_id', 'item_city', 'channel', 'finish', 'like', 'music_id', 'did', 'create_time', 'video_duration'])
print("Length of final sampled training data: ", len(train_df))
train_df.head()

Length of final sampled training data:  6582031


,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10


# Explanation of Datasets Produced in this Notebook
#### In this notebook, I take two datasets from the ICME 2019 short video recommendation competition and compile them into several new datasets to fit the purposes of this research. The final datasets are samples of the original dataset, with k-core sampling on a k value of 40. The final datasets are as follows:
1. User-Item-Ratings-Signal-1.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 4.0 If a user finished and liked a video
- 3.0 if a use liked a video but did not finish it
- 2.0 if a user finished a video but did not like it
- 1.0 if a user did not like nor finish a video

2. User-Item-Ratings-Signal-2.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 3.0 if a user finished and liked a video
- 2.0 if a user finished a video
- 1.0 otherwise

3. User-Item-Ratings-Signal-3.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 3.0 if a user finished and liked a video
- 2.0 if a user liked a video
- 1.0 if otherwise

4. User-Item-Ratings-Signal-4.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 2.0 if a user liked a video
- 1.0 if otherwise

4. User-Item-Ratings-Signal-5.csv: this is a dataset with the user, item, rating pairs, where the 'rating' is cacluated based on the following criteria: 
- 2.0 if a user finished a video
- 1.0 if otherwise

#### Additionally, in this notebook I also create a user features dataset and an item features dataset, both of which are compatible to be used with the librec-auto library for content-based filtering recommendation algorithms.

<hr/>

## Signal 1: Taking likes and finishes into consideration

In [131]:
ratings = []
rating = 0
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        like = int(features[7])
        if finish == 1 and like == 1:
            rating = 4
        elif like == 1:
            rating = 3
        elif finish == 1:
            rating = 2
        else:
            rating = 1
        ratings.append(rating)

In [132]:
train_df['signal_1'] = ratings
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,1,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,1,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,1,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


## Signal 2: Prioritizing 'finishing a video' as a positive signal

In [133]:
ratings_2 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        like = int(features[7])
        if finish == 1 and like == 1:
            rating = 3
        elif finish == 1:
            rating = 2
        else:
            rating = 1
        ratings_2.append(rating)

In [134]:
train_df['signal_2'] = ratings_2
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


## Signal 3: Prioritizing 'liking a video' as a positive signal

In [135]:
ratings_3 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        like = int(features[7])
        if finish == 1 and like == 1:
            rating = 3
        elif like == 1:
            rating = 2
        else:
            rating = 1
        ratings_3.append(rating)

In [136]:
train_df['signal_3'] = ratings_3
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


## Signal 4: Only signal is 'liking a video'

In [137]:
ratings_4 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        like = int(features[7])
        if like == 1:
            rating = 2
        else:
            rating = 1
        ratings_4.append(rating)

In [138]:
train_df['signal_4'] = ratings_4
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,1
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,1
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,1
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


## Signal 5: Only signal is 'finishing a video'

In [146]:
ratings_5 = []
with open("k_core_ratings_{0}.txt".format(k_core_file)) as fp:
    for line in fp:
        features = line.split()
        finish = int(features[6])
        if finish == 1:
            rating = 2
        else:
            rating = 1
        ratings_5.append(rating)

In [147]:
train_df['signal_5'] = ratings_5
train_df.head()

,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,did,create_time,video_duration,signal_1,signal_2,signal_3,signal_4,signal_5
0,57384,52,43192,142828,0,0,0,0,4513,34178,53085993699,39,1,1,1,1,1
1,3230,5,46822,231026,1,0,1,0,5330,24878,53086372896,16,2,2,1,1,2
2,12637,265,269351,6757,6,0,1,0,-1,27794,53086427987,4,2,2,1,1,2
3,10987,115,65804,43245,10,0,1,0,7,17481,53085562259,21,2,2,1,1,2
4,17534,123,1209085,456222,11,0,0,0,-1,34312,53086389136,10,1,1,1,1,1


<hr/>

In [148]:
train_df['signal_5'].value_counts()

1    3695472
2    2886559
Name: signal_5, dtype: int64